[comment]: # (Attach Default Lakehouse Markdown Cell)
# 📌 Attach Default Lakehouse
❗**Note the code in the cell that follows is required to programatically attach the lakehouse and enable the running of spark.sql(). If this cell fails simply restart your session as this cell MUST be the first command executed on session start.**

In [ ]:
%%configure
{
    "defaultLakehouse": {  
        "name": "lh_bronze",
    }
}

# 📦 Pip
Pip installs reqired specifically for this template should occur here

In [ ]:
# No pip installs needed for this notebook

# 🔗 Imports

In [ ]:
from notebookutils import mssparkutils # type: ignore

# #️⃣ Functions

In [ ]:
def pre_execute_notebook(notebook_file):

    try:
        mssparkutils.notebook.run(notebook_file, 1800)
        status = 'PreExecute Notebook Executed'
        error = None
    except Exception as e:
        status = 'No PreExecute Notebook Found'
        error = str(e)

    return status

def post_execute_notebook(notebook_file):

    try:
        mssparkutils.notebook.run(notebook_file, 1800)
        status = 'PostExecute Notebook Executed'
        error = None
    except Exception as e:
        status = 'No PostExecute Notebook Found'
        error = str(e)

    return status

# Pre-Execution Python Script

# Declare and Execute Pre-Execution Notebook

In [ ]:
#Read the context of the notebook
notebook_info = mssparkutils.runtime.context

# Extract the currentNotebookName from the dictionary
current_notebook_name = notebook_info.get("currentNotebookName")

In [ ]:
# Execute Pre-Execute Notebook
preexecute_notebook_name  = current_notebook_name+ ".preexecute"
pre_execute_notebook(preexecute_notebook_name)

# Declare and Execute SQL Statements

In [ ]:

sql = '''
    create or replace temporary view countries__dbt_tmp as
-- CTE to rank CDC records by Id, meta_ExtractedDate, and SYS_CHANGE_VERSION
WITH source_data AS (
    select * FROM lh_bronze.countries t1
)
select * from source_data'''

for s in sql.split(';\n'):
    spark.sql(s)


In [ ]:

sql = '''
    -- back compat for old kwarg name
  merge into lh_silver.countries as DBT_INTERNAL_DEST
      using countries__dbt_tmp as DBT_INTERNAL_SOURCE
      on 
              DBT_INTERNAL_SOURCE.CountryID = DBT_INTERNAL_DEST.CountryID
      when matched then update set
         * 
      when not matched then insert *'''

for s in sql.split(';\n'):
    spark.sql(s)


# 🛑 Execution Stop

In [ ]:
# Execute Post-Execute Notebook
postexecute_notebook_name  = current_notebook_name + ".postexecute"
post_execute_notebook(postexecute_notebook_name)

In [ ]:
#Exit to prevent spark sql debug cell running 
mssparkutils.notebook.exit("value string")

# SPARK SQL Cells for Debugging

In [ ]:
%%sql
    create or replace temporary view countries__dbt_tmp as
-- CTE to rank CDC records by Id, meta_ExtractedDate, and SYS_CHANGE_VERSION
WITH source_data AS (
    select * FROM lh_bronze.countries t1
)
select * from source_data

In [ ]:
%%sql
    -- back compat for old kwarg name
  merge into lh_silver.countries as DBT_INTERNAL_DEST
      using countries__dbt_tmp as DBT_INTERNAL_SOURCE
      on 
              DBT_INTERNAL_SOURCE.CountryID = DBT_INTERNAL_DEST.CountryID
      when matched then update set
         * 
      when not matched then insert *